# 客户端

演示客户端如何与简单的链服务器交互，该服务器部署了一个关于特定主题讲笑话的链。

您可以通过API直接与此交互。

In [1]:
import requests

inputs = {"input": {"topic": "sports"}}
response = requests.post("http://localhost:8000/configurable_temp/invoke", json=inputs)

response.json()

{'output': '为什么篮球运动员总是带那么多钱？\n\n因为他们总是被罚球！ 🏀 😂 \n\n\n你喜欢这个笑话吗？你想听另一个吗？\n',
 'metadata': {'run_id': '92ff751e-9287-4f2b-9859-3d8d0a7a6999',
  'feedback_tokens': []}}

您也可以通过RemoteRunnable接口与之交互（用于其他链中）。

In [2]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/configurable_temp")

远程可运行实例与本地可运行实例具有相同的接口。

In [3]:
response = await remote_runnable.ainvoke({"topic": "sports"})

客户端还可以同步执行LangChain代码，并传入配置。

In [4]:
from langchain.schema.runnable.config import RunnableConfig

remote_runnable.batch([{"topic": "sports"}, {"topic": "cars"}])

['为什么足球运动员总是带着钱去睡觉？\n\n因为他们害怕睡梦中会被球门守住！ \n\n\n希望你喜欢这个笑话！ 😄\n',
 '为什么汽车总是那么紧张？\n\n因为它们担心会“刹车”！ \n\n\n']

服务器支持流式传输（使用HTTP服务器端事件），这有助于实时与长响应进行交互。

In [5]:
async for chunk in remote_runnable.astream({"topic": "bears, but a bit verbose"}):
    print(chunk, end="", flush=True)

在一个寒冷的冬日，一只名叫巴里的小熊在森林里漫步。他肚子饿得咕咕叫，可是他找不到任何食物。

“天哪，”巴里叹了口气，“我真是太饿了！我该怎么办呢？”

这时，一只老滑稽的松鼠从树枝上跳下来，好奇地盯着巴里。

“嘿，小熊，”松鼠说道，“你看起来很愁眉苦脸。有什么事吗？”

巴里告诉了松鼠他的困境，然后问道：“你知道哪里能找到吃的吗？”

松鼠想了想，然后笑着说：“我知道一个地方，那里有大量的蜂蜜！但是，你必须穿过一片荆棘丛林才能到达那里。”

巴里犹豫了一下，他很害怕荆棘丛林，那里充满了危险。但他实在太饿了，于是决定冒险一试。

“好，”巴里说，“我愿意去尝试！”

松鼠指了指一条小路，说：“沿着这条小路走，你就能找到荆棘丛林的入口。”

巴里沿着小路走，越走越感觉不安。他看到地上散落着许多动物的毛发，还有几只被荆棘扎伤的鸟儿。

“这片荆棘丛林看起来真可怕，”巴里低声说道。

他鼓起勇气，踏进了荆棘丛林。

荆棘丛林里到处都是尖刺和荆条，巴里被扎得浑身是血。他艰难地前行，每一步都像走进了火炉。

终于，他看到了尽头，那里有一棵巨大的橡树，树上挂着满满的蜂蜜。

巴里兴奋地跑过去，想尽力地舔着蜂蜜。

这时，松鼠突然从树枝上跳下来，对着巴里大声喊道：“哈哈！我骗了你！这只是个玩笑！”

巴里被松鼠的恶作剧吓了一跳，他愤怒地盯着松鼠。

“你为什么这样做？”巴里问道，“你害我受了伤！”

松鼠狡黠地笑着说：“我只是想和你开个玩笑！你太容易被骗了！”

巴里气得浑身发抖，但他知道自己无法战胜松鼠。他只能默默地离开，带着伤痛和愤怒。

从此以后，巴里再也不敢相信松鼠的话了。他学会了要小心谨慎，不要轻易相信任何陌生人的话。




## 可配置性

服务器链已被公开为可配置链！

```python
model = ChatOpenAI(temperature=0.5).configurable_alternatives(
    ConfigurableField(
        id="llm",
        name="语言模型",
        description=(
            "决定是否为语言模型使用高或低的温度参数。"
        ),
    ),
    high_temp=ChatOpenAI(temperature=0.9),
    low_temp=ChatOpenAI(temperature=0.1),
    default_key="medium_temp",
)
prompt = PromptTemplate.from_template(
    "讲一个关于{topic}的笑话。"  # 可配置字段的示例
).configurable_fields(
    template=ConfigurableField(
        id="prompt",
        name="提示",
        description="要使用的提示。必须包含{topic}。",
    )
)
```

我们现在可以在API中使用可运行实例的可配置性了！

In [11]:
await remote_runnable.ainvoke(
    {"topic": "运动"},
    config={
        "configurable": {"prompt": "如何用法语表达{topic}", "llm": "low_temp"}
    },
)

"用法语表达“运动”，可以根据具体语境选择不同的表达方式：\n\n**1.  Faire du sport:** 这是最常见的表达方式，意为“做运动”。\n\n* **例句:** Je fais du sport trois fois par semaine. (我每周运动三次。)\n\n**2. S'entraîner:**  意为“训练”，通常指有针对性的运动，例如为了比赛或达到特定目标。\n\n* **例句:** Il s'entraîne dur pour le marathon. (他为马拉松比赛进行着艰苦的训练。)\n\n**3. Bouger:**  意为“活动”，可以指任何类型的运动，包括日常活动。\n\n* **例句:** Il faut bouger plus! (要多活动一下!)\n\n**4. Se déplacer:**  意为“移动”，通常指身体的移动，可以指步行、跑步、骑自行车等。\n\n* **例句:** Comment se déplacer en ville? (如何在城市里出行?)\n\n**5. Pratiquer un sport:**  意为“练习一种运动”，指特定类型的运动。\n\n* **例句:** Je pratique le tennis depuis mon enfance. (我从小就练习网球。)\n\n**6. Jouer à un sport:**  意为“玩一种运动”，通常指团队运动。\n\n* **例句:** Ils jouent au football tous les dimanches. (他们每个星期天都踢足球。)\n\n**7. Faire de l'exercice:**  意为“做运动”，通常指室内运动，例如瑜伽、健身等。\n\n* **例句:** Je fais de l'exercice à la maison. (我在家做运动。)\n\n**8. Se mettre en forme:**  意为“保持健康”，指通过运动来改善身体状况。\n\n* **例句:** Je veux me mettre en forme pour l'été. (我想在夏天保持健康。)\n\n\n\n希望这些表达方式能帮到你！\n"

## 基于请求属性的可配置性

如果你想根据请求的属性来改变链的调用方式，可以使用`add_routes`的`per_req_config_modifier`方法，如下所示：

```python
# 添加另一个示例路由，你可以根据请求的属性来配置模型。
# 这适用于从请求头传递API密钥（需要谨慎）或使用请求的其他属性来配置模型。
def fetch_api_key_from_header(config: Dict[str, Any], req: Request) -> Dict[str, Any]:
    if "x-api-key" in req.headers:
        config["configurable"]["openai_api_key"] = req.headers["x-api-key"]
    return config

dynamic_auth_model = ChatOpenAI(openai_api_key='placeholder').configurable_fields(
    openai_api_key=ConfigurableField(
        id="openai_api_key",
        name="OpenAI API 密钥",
        description=(
            "用于OpenAI交互的API密钥"
        ),
    ),
)

dynamic_auth_chain = dynamic_auth_model | StrOutputParser()

add_routes(
    app, 
    dynamic_auth_chain, 
    path="/auth_from_header",
    config_keys=["configurable"], 
    per_req_config_modifier=fetch_api_key_from_header
)
```

现在，我们可以看到，只有设置了特定的请求头(request header)，我们的模型请求才能正常工作：

In [12]:
# 模型将因认证错误而失败
unauthenticated_response = requests.post(
    "http://localhost:8000/auth_from_header/invoke", json={"input": "你好"}
)
unauthenticated_response.json()

{'detail': '未提供API密钥'}

现在，请确保你已经在本地的命令行上运行了以下命令：
```bash
export TEST_API_KEY=<在这里插入我的密钥>
```

In [10]:
# 只要之前运行了上述脚本，模型就会成功
import os

test_key = os.environ["TEST_API_KEY"]
authenticated_response = requests.post(
    "http://localhost:8000/auth_from_header/invoke",
    json={"input": "你好"},
    headers={"x-api-key": test_key},
)
authenticated_response.json()